# read_write_lib.py

In [64]:
# import cv2
import numpy as np

def image_read_ppm(dirFile):
    f = open(dirFile, 'r')
    header_1 = f.readline()
    header_2 = f.readline()
    header_3 = f.readline()

    IMG_WIDTH  = int(header_2.split()[0])
    IMG_HEIGHT = int(header_2.split()[1])

    header = np.array([header_1, header_2, header_3])
    data = f.read().replace("\r", " ").replace("\n", " ")
    dataTmp = data.split()
    dataList = dataTmp[:IMG_HEIGHT*IMG_WIDTH*3]
    outData = np.array(dataList, dtype=int).reshape(IMG_HEIGHT, IMG_WIDTH, 3)

    return outData, IMG_WIDTH, IMG_HEIGHT, header

def image_write_ppm(dirFile, imgData, header):
    header_1 = header[0]
    header_2 = header[1]
    header_3 = header[2]

    IMG_WIDTH  = int(header_2.split()[0])
    IMG_HEIGHT = int(header_2.split()[1])

    with open(dirFile, 'w') as f:
        f.write(header_1)
        f.write(header_2)
        f.write(header_3)
        for i in range(IMG_HEIGHT):
            for j in range(IMG_WIDTH):
                f.write(str(imgData[i,j,0]) + " " )
                f.write(str(imgData[i,j,1]) + " " )
                f.write(str(imgData[i,j,2]) + "\n")

# fixed_point_lib.py

In [61]:
import numpy as np

def func_round_diff(idata):
    odata = np.where(np.absolute(idata - idata.astype(np.int32)) >= 0.5, np.sign(idata) * (abs(idata).astype(np.int32)+1), idata.astype(np.int32))
    return odata

def round_array(idata, rl):
    odata = func_round_diff(idata/(2**rl))
    return odata

def fixed_pts_array(idata, fl):
    odata = func_round_diff(idata*(2**fl))
    return odata

# math.py

In [127]:
def cov(input_data, filter, pad=2):
    H, W = input_data.shape
    f_H, f_W = filter.shape

    output_H = (H + 2 * pad - f_H) + 1
    output_W = (W + 2 * pad - f_W) + 1

    pad_data = np.pad(input_data, [(pad, pad), (pad, pad)], 'edge')
    output = np.zeros((output_H, output_W))

    for h in range(H):       
        for w in range(W):
            output[h, w] = np.sum(pad_data[h:h+f_H, w:w+f_W] * filter)
            
    return output

def mat_mul(input_data, )


# Main Code

In [128]:
import numpy as np

if __name__ == "__main__":
    #############################
    # 1. Parameter setting
    #############################
    PPM_NAME = "ONE_DOT.ppm"
    INPUT_IMAGE_DIR = "./image/input/"
    OUTPUT_IMAGE_DIR = "./image/output/"

    INPUT_IMAGE_FILE_PATH = INPUT_IMAGE_DIR + PPM_NAME
    OUTPUT_IMAGE_FILE_PAHT = OUTPUT_IMAGE_DIR + PPM_NAME

    imgData, IMG_WIDTH, IMG_HEIGHT, header = image_read_ppm(INPUT_IMAGE_FILE_PATH)
    print(imgData.shape)
    r = imgData[:,:,0]
    print(r.shape)
    # image_write_ppm(OUTPUT_IMAGE_FILE_PAHT, imgData, header)

    FILTER_SIZE = 5
    PADDING_SIZE = int(FILTER_SIZE/2)

    mean_filter_coef = np.ones((5,5)) / (FILTER_SIZE*FILTER_SIZE-1)
    mean_filter_coef[2,2] = 0.5
    sharpness_filter_coef = -np.ones((5,5)) / (FILTER_SIZE*FILTER_SIZE - 1)

    mean_filter_coef_fixed = fixed_pts_array(mean_filter_coef, 8)
    #############################
    # 2. 
    #############################
    X = np.asarray(
    [
    [1,
     5,
     4,
     2,
     6],

    [3,
     5,
     6,
     9,
     1]
    ]
    )

    print(X)
    print(mean_filter_coef_fixed)
    X = cov(X, mean_filter_coef_fixed)

    print(X)

(1080, 1920, 3)
(1080, 1920)
[[1 5 4 2 6]
 [3 5 6 9 1]]
[[ 11.  11.  11.  11.  11.]
 [ 11.  11.  11.  11.  11.]
 [ 11.  11. 128.  11.  11.]
 [ 11.  11.  11.  11.  11.]
 [ 11.  11.  11.  11.  11.]]
[[ 953. 1586. 1590. 1477. 1890.]
 [1275. 1729. 1890. 2285. 1239.]]
